In [1]:
import os
import getpass
from typing import Annotated, List
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_chroma import Chroma
from langchain_core.tools import BaseTool
from langgraph.prebuilt import ToolNode, tools_condition

ModuleNotFoundError: No module named 'langchain_chroma'

In [ ]:
class MultiState(TypedDict):
  messages: Annotated[list, add_messages]

vectorstore = None

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = init_chat_model("google_genai:gemini-2.0-flash")

In [ ]:
def setup_pdfs( pdf_folder: str = "pdfs" ):
  global vectorstore
  loader = DirectoryLoader(pdf_folder, glob="**/*.pdfs", loader_cls=PyPDFLoader)
  docs = loader.load()
  chunks = splitter.split_documents(docs)
  vectorstore = FAISS.from_documents(chunks, embedding)

In [ ]:
def search_pdfs_function(query: str) -> str:
  docs = vectorstore.as_retriever().invoke(query)
  return docs

In [ ]:
class PDFSearchTool(BaseTool):
  name: str = "searchPDFs"
  description: str = "Search PDF for Document Information"
  def _run(self, query : str) -> str:
    return search_pdfs_function(query)

In [ ]:
tool = PDFSearchTool()
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def chatbot(state: MultiState):
  return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [ ]:
graph_builder = StateGraph(MultiState)

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges(
  'chatbot',
  tools_condition
)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [ ]:
def stream_graph_updates(user_input: str):
  for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
    for value in event.values():
      print("Assistant: ", value['messages'][-1].content)

In [ ]:
def show_all_pdf_content(docs):
    """Show all PDF content"""
    print("\n" + "="*50)
    print("📚 ALL PDF CONTENT")
    print("="*50)
    
    for i, doc in enumerate(docs):
        print(f"\n--- Document {i+1} ---")
        print(f"Source: {doc.metadata.get('source', 'Unknown')}")
        print(f"Page: {doc.metadata.get('page', 'Unknown')}")
        print(f"Content:\n{doc.page_content}")
        print("-" * 30)
print("🚀 Loading PDFs...")
docs, chunks = setup_pdfs()

show_all_pdf_content(docs)

In [ ]:
while True:
  user_input = input("User: ")
  if user_input.lower() in ["quit", "exit", "by"]:
    print("Good By")
    break
  stream_graph_updates(user_input)